In [7]:
import numpy as np
import queue
from mf.mf_func import MFOptFunction
from utils.general_utils import map_to_cube
import sys
from examples.synthetic_functions import *
from mf.mf_func import get_noisy_mfof_from_mfof
from letters.letters_classifier import *
import time
from MFTreeSearchCV.POO import *

import synthetic_functions

ModuleNotFoundError: No module named '__main__.mf'; '__main__' is not a package

In [3]:
def branin_function(x, a, b, c, r, s, t):
  """ Computes the Branin function. """
  x1 = x[0]
  x2 = x[1]
  neg_ret = a * (x2 - b*x1**2 + c*x1 - r)**2 + s*(1-t)*np.cos(x1) + s
  return -neg_ret

def branin_function_alpha(x, alpha, a, r, s):
  """ Alternative form for the branin function. """
  return branin_function(x, a, alpha[0], alpha[1], r, s, alpha[2])

def get_mf_branin_function(fidel_dim):
  """ Returns the Branin function as a multifidelity function. """
  a0 = 1
  b0 = 5.1/(4*np.pi**2)
  c0 = 5/np.pi
  r0 = 6
  s0 = 10
  t0 = 1/(8*np.pi)
  alpha = np.array([b0, c0, t0])
  # Define delta
  delta = [0.01, 0.1, -0.005]
  delta = np.array(delta[0:fidel_dim] + [0] * (3 - fidel_dim))

  def mf_branin_obj(z, x):
    """ Wrapper for the MF Branin objective. """
    assert len(z) == fidel_dim
    z_extended = np.append(z, [0] * (3-fidel_dim))
    alpha_z = alpha - (1 - z_extended) * delta
    return branin_function_alpha(x, alpha_z, a0, r0, s0)
  # Other data
  opt_fidel = np.ones((fidel_dim))
  fidel_bounds = [[0, 1]] * fidel_dim
  opt_pt = np.array([np.pi, 2.275])
  domain_bounds = [[-5, 10], [0, 15]]
  return mf_branin_obj, opt_pt, opt_fidel, fidel_bounds, domain_bounds

def get_mf_branin_as_mfof(fidel_dim):
  """ Wrapper for get_mf_branin_function which returns as a mfof. """
  mf_branin_obj, opt_pt, opt_fidel, fidel_bounds, domain_bounds = \
    get_mf_branin_function(fidel_dim)
  fidel_cost_function = _get_mf_cost_function(fidel_bounds, True)
  opt_val = mf_branin_obj(opt_fidel, opt_pt)
  return MFOptFunction(mf_branin_obj, fidel_cost_function, fidel_bounds, domain_bounds,
                       opt_fidel, vectorised=False, opt_pt=opt_pt, opt_val=opt_val)

In [9]:
NUM_EXP = 5
EXP_NAME = 'Branin'
#EXP_NAME = 'CurrinExp'
#EXP_NAME = 'Hartmann3'
#EXP_NAME = 'Hartmann6'
#EXP_NAME = 'Borehole'


def run_one_experiment(mfobject,nu,rho,times,sigma,C,t0,filname):
	R = []
	T = []
	for t in times:
		budget = t*mfobject.opt_fidel_cost
		t1 = time.time()
		MP = POO(mfobject=mfobject, nu_max=nu, rho_max=rho, total_budget=budget, sigma=sigma, C=C, mult=0.5, tol = 1e-3, Randomize = False, Auto = False, unit_cost=t0 )
		MP.run_all_MFHOO()
		X, E = MP.get_point()
		t2 = time.time()	

		R = R + [E]
		T = T + [MP.cost]
		#print 'Total HOO Queries: ' + str(MP.t) 
		print(str(MP.cost) + ' : ' + str(E))

	np.save(filename,R)
	return np.array(R),np.array(T)


In [10]:
if EXP_NAME == 'Hartmann3':
    mfof = synthetic_functions.get_mf_hartmann_as_mfof(1, 3)
    noise_var = 0.01
    sigma = np.sqrt(noise_var)

elif EXP_NAME == 'Hartmann6':
    mfof = synthetic_functions.get_mf_hartmann_as_mfof(1, 6)
    max_capital = 200 * mfof.opt_fidel_cost
    noise_var = 0.05
    sigma = np.sqrt(noise_var)
elif EXP_NAME == 'CurrinExp':
    mfof = synthetic_functions.get_mf_currin_exp_as_mfof()
    max_capital = 200 * mfof.opt_fidel_cost
    noise_var = 0.5
    sigma = np.sqrt(noise_var)
elif EXP_NAME == 'Branin':
    mfof = synthetic_functions.get_mf_branin_as_mfof(1)
    max_capital = 200 * mfof.opt_fidel_cost
    noise_var = 0.05
    sigma = np.sqrt(noise_var)
elif EXP_NAME == 'Borehole':
    mfof = synthetic_functions.get_mf_borehole_as_mfof()
    max_capital = 200 * mfof.opt_fidel_cost
    noise_var = 5
    sigma = np.sqrt(noise_var)

NameError: name 'synthetic_functions' is not defined